### Basic Model

In [3]:
# from langchain.llms.openai import OpenAI # text-davinci-003 Model (사용 안 함)
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
)

C:\Users\Public\Temporary\TEMP\ipykernel_101792\2288673006.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [3]:
##### 방식 1. HumanMessage, AIMessage, SystemMessage 를 직접 묶어서 messages로 사용 #####
from langchain.schema import HumanMessage, AIMessage, SystemMessage 


# HumanMessage: 사용자 입력, AIMessage: AI 출력, SystemMessage: 시스템 출력, prompt를 직접 입력할 때 사용

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in {language}."), # SystemMessage(content=AI의 설정값 등을 입력함)
    AIMessage(content="Ciao, mi chiamo {name}!"), # AIMessage(content=AI의 출력값 등을 입력함)
    HumanMessage(content="What is the distance between {country_a} and {country_b}? Also, what is your name?") # HumanMessage(content=사용자 입력값)
]

chat.predict_messages(messages)

AIMessage(content="Il mio nome è {name}! Mi dispiace, non posso calcolare la distanza esatta tra due paesi senza informazioni specifiche sulla posizione esatta all'interno di ciascun paese. Potresti fornirmi ulteriori dettagli? \n\n번역: 제 이름은 {name}입니다! 죄송합니다만, 두 나라 사이의 정확한 거리를 계산하기 위해서는 각 나라 내의 정확한 위치 정보가 필요합니다. 추가 정보를 제공해 주시겠어요?")

결과값:
AIMessage(content='Mi dispiace, non posso fornire informazioni sulla distanza tra due paesi specifici. Il mio nome è {name}. Posso aiutarti con altre domande geografiche?')
<br/> 뜻: 죄송합니다만, 특정한 두 나라 사이의 거리에 대한 정보는 말씀드릴 수가 없습니다. 나의 이름은 {name}이다. 다른 지리적 질문도 도와드릴까요?

message에서 {language}, {name}, {country_a}, {country_b}가 설정되어 있지 않아 제대로 된 정보가 출력되지 않음.

In [3]:
##### 방식 2. PromptTemplate 사용 #####
from langchain.prompts import PromptTemplate # PromtTemplate은 string에서 template을 생성하기 위해 사용

template_c = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}? Also, what is your name?") # template 생성, {country_a}, {country_b}는 변수

prompt_a = template_c.format(country_a="Maxico", country_b="Thailand") # template에 변수를 적용하여 prompt 생성

chat.predict(prompt_a) # prompt_a를 입력하여 AI 출력 예측

NameError: name 'chat' is not defined

결과값:
'The distance between Mexico and Thailand is approximately 15,000 kilometers. My name is Assistant.'

In [2]:
##### 방식 3. ChatPromptTemplate 사용 #####
from langchain.prompts import ChatPromptTemplate # CharPromptTemplate는 messages에서 template을 생성하기 위해 사용

template_b = ChatPromptTemplate.from_messages( # system, ai, human 순서로 messages를 ("speaker", "content") 형태로 입력, {language}, {name}, {country_a}, {country_b}는 변수
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}? Also, what is your name?")
    ]
)

prompt_b = template_b.format_messages( # template에 변수를 적용하여 prompt 생성
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt_b) # prompt_b를 입력하여 AI 출력 예측

NameError: name 'chat' is not defined

결과값: AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')
<br/> 뜻: 안녕하세요! 내 이름은 소크라테스입니다. 멕시코와 태국 간의 거리는 약 16,000km입니다.

In [6]:
##### 방식 4. ChatPromptTemplate 사용 및 사용자 정의 함수를 사용한 응용 #####
from langchain.schema import BaseOutputParser

template_c = ChatPromptTemplate.from_messages( 
    [
        # {max_items}은 답변의 최대 개수
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lower case. Do NOT reply with anything else."),
        # {question}은 사용자의 질문
        ("human", "{question}")
    ]
)

# template에 변수를 적용하여 prompt 생성
prompt_c = template_c.format_messages( 
    max_items=5,
    question="What are the planets?"
)

# prompt_c를 입력하여 AI 출력 예측
result = chat.predict_messages(prompt_c) 


# AI 출력을 파싱하기 위한 클래스, BaseOutputParser를 상속받아 구현, 파싱이란? AI 출력을 의미있는 형태로 변환하는 것
class CommaOutputParser(BaseOutputParser): 
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

par = CommaOutputParser() # CommaOutputParser 객체 par 생성 (함수를 사용하기 위해 생성 필요함)
par.parse(result.content)

['mercury', 'venus', 'earth', 'mars', 'jupiter']

(max_items = 10) 결과값: <br/>
['mercury','venus','earth','mars','jupiter','saturn','uranus','neptune','pluto']

(max_items = 5) 결과값: <br/>
['mercury','venus','earth','mars','jupiter']

In [7]:
##### 방식 5. Langchain format을 사용한 응용 #####

# template -> chat -> par 순서로 실행됨
chain = template_c | chat | par 

chain.invoke(
    {
        "max_items": 10,
        "question": "What are the pokemons?"
    }
)

# 이런 방식으로도 조합할 수 있음
# chain_one = template1 | chat1 | parser1: template1 -> chat1 -> parser1 순서로 실행됨
# chain_two = template2 | chat2 | parser2: template2 -> chat2 -> parser2 순서로 실행됨
# all = chain_one | chain_two | ouput: chain_one -> chain_two -> output 순서로 실행됨

['bulbasaur',
 'ivysaur',
 'venusaur',
 'charmander',
 'charmeleon',
 'charizard',
 'squirtle',
 'wartortle',
 'blastoise',
 'pikachu']

(max_items = 10) 결과값: <br/>
['bulbasaur', 'ivysaur', 'venusaur', 'charmander', 'charmeleon', 'charizard', 'squirtle', 'wartortle', 'blastoise', 'pikachu']

### LangChain?
template1 | chat1 | parser1 로 짜여진 chain을 invoke 할 경우
<br/> 첫 번째 input이 template1로 입력되어 첫 번째 output이 되어 결과로 나오고
<br/> 해당 output1이 chat1로 입력되어 두 번째 output이 되어 결과로 나오고
<br/> 해당 output2가 parser1로 입력되어 최종 결과로 나옴 

##### 5 Input Types of LangChain
1. Prompt: Dict Type
2. Retriever: String Type
3. LLM: String Type
    3-1. ChatModel: List Type (chat message나 PromptValue Type 자료를 담고 있음)
4. Tool: String Type
5. OutputParser

##### Output Types of LangChain
1. LLM: String Type
    1-1. ChatModel: ChatMessage Type
2. Prompt: PromptValue Type
3. Retriever: List of Documents
4. Tool: Depends on the tool
5. OutputParser: Depends on the parser

---

In [8]:
# from langchain.llms.openai import OpenAI # text-davinci-003 Model (사용 안 함)
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
    ,streaming=True # streaming=True로 설정하면, AI 출력이 나올 때마다 출력을 반환함, 모든 출력이 다 나올때까지 대기할 필요가 없음
    ,callbacks=[StreamingStdOutCallbackHandler()] # streaming=True로 설정하면, 출력을 받을 때마다 callback 함수를 호출함
)

In [9]:
# {요리 이름}을 입력하면 요리법을 알려주는 챗봇
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipe for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

# chef_prompt에 {cuisine}를 넣어 prompt를 생성하고, system 메시지를 수행하는 chain 생성
chef_chain = chef_prompt | chat

<img src=".\assets\images\Result1.png">

In [13]:
# {요리법}을 입력하면 해당 요리법의 재료를 Vegetarian 용으로 바꿔주는 챗봇
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])
# "If there is no alternative for a food just say you don't know how to replace it." Hallucination 방지를 위한 문장

# veg_chef_prompt에 {recipe}를 넣어 prompt를 생성하고, system 메시지를 수행하는 chain 생성
veg_chain = veg_chef_prompt | chat

<img src=".\assets\images\Result2.png">

In [15]:
# chain model을 연계하여 사용하지 않을 경우 chef_chain, veg_chain을 각각 사용해야 함
# chef_chain.invoke({"cuisine": "indian"}) -> "Chicken Tikka Masala"
# veg_chain.invoke({"recipe": "Chicken Tikka Masala"}) -> veg_recipe of "Chicken Tikka Masala"

# 반면 chain model을 연계하여 사용할 경우 chef_chain, veg_chain을 각각 사용하지 않아도 됨
# final_chain = chef_chain | veg_chain -> 이 경우 chef_chain의 결과값이 veg_chain의 {recipe}에 들어갈 수 없음, 위치를 명확히 지정해줘야 들어감
final_chain = {"recipe": chef_chain} | veg_chain # chef_chain의 결과값이 veg_chain의 {recipe}에 들어감
final_chain.invoke({
    "cuisine": "indian" #cuisine이 chef_chain의 {cuisine}에 들어감
})

Great choice! How about trying to make a classic Indian dish like Chicken Tikka Masala? Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground cinnamon
- 1 teaspoon ground turmeric
- 1 teaspoon ground coriander
- 1 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon grated ginger
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, cinnamon, turmeric, coriander, cayenne pepper, salt, and pepper. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°F (200°C). 

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (use plant-based yogurt for a vegan version)\n- 2 tablespoons lemon juice\n- 2 teaspoons ground cumin\n- 2 teaspoons paprika\n- 1 teaspoon ground cinnamon\n- 1 teaspoon ground turmeric\n- 1 teaspoon ground coriander\n- 1 teaspoon cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon grated ginger\n- 1 can (14 oz) tomato sauce\n- 1 cup coconut cream (or cashew cream for a lighter option)\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but use tofu or paneer instead of chicken. Marinate the tofu or paneer in the yogurt and spic

<img src=.\assets\images\Result3.png>

#### 1 차 Chicken Tikka Masala에 대한 Recipe 출력 (실제로 출력되지 않음)
Great choice! How about trying to make a classic Indian dish like Chicken Tikka Masala? Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground cinnamon
- 1 teaspoon ground turmeric
- 1 teaspoon ground coriander
- 1 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon grated ginger
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, cinnamon, turmeric, coriander, cayenne pepper, salt, and pepper. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°F (200°C). Thread the marinated chicken pieces onto skewers and place them on a baking sheet. Bake for 20-25 minutes, or until the chicken is cooked through.

3. In a large skillet, heat the vegetable oil over medium heat. Add the chopped onion and cook until softened, about 5 minutes. Add the garlic and ginger, and cook for another minute.

4. Stir in the tomato sauce and bring to a simmer. Add the baked chicken pieces to the sauce and simmer for 10 minutes.

5. Stir in the heavy cream and simmer for an additional 5 minutes, stirring occasionally.

6. Serve the Chicken Tikka Masala over steamed rice, garnished with fresh cilantro. Enjoy your delicious homemade Indian meal!


#### 2 차 Chicken Tikka Masala에 대한 Recipe를 Vegeterian에 맞춰 변환하여 출력 (최종 출력물)
Feel free to adjust the spice levels to suit your taste preferences. Enjoy your culinary adventure in making Chicken Tikka Masala!For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:

Ingredients:
- 1 lb firm tofu or paneer, cut into bite-sized pieces
- 1 cup plain yogurt (use plant-based yogurt for a vegan version)
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground cinnamon
- 1 teaspoon ground turmeric
- 1 teaspoon ground coriander
- 1 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon grated ginger
- 1 can (14 oz) tomato sauce
- 1 cup coconut cream (or cashew cream for a lighter option)
- Fresh cilantro, chopped (for garnish)

Instructions:
1. Follow the same marinating process as the original recipe, but use tofu or paneer instead of chicken. Marinate the tofu or paneer in the yogurt and spice mixture for at least 1 hour.

2. Instead of baking the chicken, you can pan-fry the marinated tofu or paneer until golden brown on all sides.

3. Proceed with the recipe as instructed, replacing the chicken with the cooked tofu or paneer when adding it to the tomato sauce.

4. Substitute heavy cream with coconut cream or cashew cream for a creamy texture in the dish.

5. Serve the Vegetarian Tikka Masala over steamed rice, garnished with fresh cilantro.

By making these simple ingredient swaps, you can enjoy a delicious vegetarian version of Chicken Tikka Masala that retains the flavors of the traditional dish. Enjoy your culinary adventure!

---